# Synapses plots

Generate plots to show the distribution of pre- and post-synaptic terminals in an example annotated neuron.

In [ ]:
%matplotlib widget

import numpy as np
from lotr import DATASET_LOCATION
from lotr import plotting as pltltr
from matplotlib import pyplot as plt
from tifffile import imread

COLS = pltltr.COLS

In [ ]:
isynapses_file = DATASET_LOCATION / "anatomy" / "em_reconstructions" / "synapses.xml"

import xml.etree.ElementTree as et

from lotr.em.core import EmNeuron
from lotr.em.loading import load_skeletons_from_zip

path = str(isynapses_file )
xtree = et.parse(path)
node = xtree.getroot()
synapses = []
for n in list(node):  # ugly loop
    if n.tag == "thing":
        synapses.append(EmNeuron(n))

# Load the EM and mirror neurons on one side:
em_path = DATASET_LOCATION / "anatomy" / "em_reconstructions" / "aHB_dIPN.k.zip"

neurons_list = load_skeletons_from_zip(em_path)
sel_neuron_id = "c008"

neuron = [n for n in neurons_list if n.id == sel_neuron_id][0]

In [ ]:
from bg_atlasapi.core import Atlas

atlas = Atlas(DATASET_LOCATION / "anatomy" / "ipn_zfish_0.5um_v1.8")

#  Specify axes limits over all dimensions:
bs = dict(frontal=(30, 180), vertical=(-4, 170), sagittal=(-50, 120))

plotter = pltltr.AtlasPlotter(
    atlas=atlas,
    structures=["ipn", "dipn"],
    mask_slices=dict(frontal=slice(0, 120)),
    bounds_dict=dict(
        frontal=[bs["vertical"], bs["frontal"]],
        horizontal=[bs["sagittal"], bs["frontal"]],
        sagittal=[bs["vertical"], bs["sagittal"]],
    ),
)

In [ ]:
lw = 0.8
f, axs = plotter.generate_projection_plots()
main_c = ".5" 

# Plot axon and dendrites in two different colors:
for select, shift in zip(["dendrites", "axon"], [-0.05, 0.2]):
    plotter.plot_neurons(
        axs,
        neuron,
        select=select,
        c=pltltr.shift_lum(main_c, shift),
        lw=lw,
        soma_s=30 if select=="dendrites" else 0,
        label="__nolegend__",
    )

# scatterplot of synapses in different colors::
for syn, lab, col in zip(
    synapses, ["presyn. terminals", "postsyn. terminals"], ["C2", "C6"]
):
    plotter.axs_scatterplot(
        axs, syn.coords_ipn, color=col, s=1, label="__nolabel__", zorder=100
    )
    plotter.axs_scatterplot(
        axs, np.ones((1, 3)) * 1000, color=col, s=20, label=lab, zorder=100
    )  # dummy for legend

axs[1].set_title(neuron.id)
axs[1].legend(handlelength=0.0)

In [ ]:
pltltr.savefig("synapses_plot", folder="S11b")